In [ ]:
import requests
from bs4 import BeautifulSoup
import smtplib
import time
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Job search parameters
keywords = ["data scientist", "data engineer", "data analyst"]
location = "remote"
job_boards = [
    "https://www.indeed.com/jobs?q={}&l={}",
    "https://www.linkedin.com/jobs/search?keywords={}&location={}",
    "https://www.glassdoor.com/Job/jobs.htm?sc.keyword={}&locT=C&locId=1&locKeyword={}",
    "https://remoterocketship.com/jobs?q={}&l={}",
]

# Function to scrape job postings
def scrape_jobs():
    job_list = []
    for board in job_boards:
        for keyword in keywords:
            url = board.format(keyword.replace(" ", "+"), location.replace(" ", "+"))
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                if 'indeed' in url:
                    jobs = soup.find_all('a', class_='tapItem')
                    for job in jobs:
                        title = job.find('h2').text.strip()
                        link = "https://www.indeed.com" + job.get('href')
                        job_list.append((title, link))
                
                elif 'linkedin' in url:
                    jobs = soup.find_all('div', class_='base-search-card')
                    for job in jobs:
                        title = job.find('h3').text.strip()
                        link = job.find('a').get('href')
                        job_list.append((title, link))
                
                elif 'glassdoor' in url:
                    jobs = soup.find_all('li', class_='react-job-listing')
                    for job in jobs:
                        title = job.find('a').text.strip()
                        link = "https://www.glassdoor.com" + job.find('a').get('href')
                        job_list.append((title, link))
                
                elif 'remoterocketship' in url:
                    jobs = soup.find_all('div', class_='job-card')
                    for job in jobs:
                        title = job.find('h2').text.strip()
                        link = job.find('a').get('href')
                        job_list.append((title, link))
                
    return job_list

# Function to send email alerts
def send_email(job_list):
    sender_email = "your_email@gmail.com"
    receiver_email = "your_email@gmail.com"
    password = "your_email_password"

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = "New Remote Data Job Openings"
    
    body = "Here are the latest job postings:\n\n"
    for job in job_list:
        body += f"{job[0]} - {job[1]}\n"
    
    msg.attach(MIMEText(body, 'plain'))
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()

# Running the script
if __name__ == "__main__":
    while True:
        jobs = scrape_jobs()
        if jobs:
            df = pd.DataFrame(jobs, columns=['Job Title', 'Job Link'])
            df.to_csv('job_listings.csv', index=False)
            send_email(jobs)
        time.sleep(86400)  # Runs daily
